# Loading Libraries

In [1]:
import pandas as pd
import numpy as np

import surprise
from recommenders.models.surprise.surprise_utils import predict, compute_ranking_predictions
from recommenders.evaluation.python_evaluation import precision_at_k, recall_at_k


# Reading Data

In [2]:
# Read Train & Test Data
train = pd.read_csv("../../00_Data/rec_sys_ratings_train.csv", index_col=0)
test = pd.read_csv("../../00_Data/rec_sys_ratings_test.csv", index_col=0)

In [3]:
# Check Shapes
print("Shape of Train:", train.shape)
print("Shape of Test: \t", test.shape)

Shape of Train: (393208, 3)
Shape of Test: 	 (98401, 3)


In [4]:
# Check Head of Train
train.head()

,StockCode,CustomerID,purchased
444302,22326,14323,-1
1750,16169E,16607,1
411656,23208,15147,-1
302355,23301,12580,-1
7870,20829,17888,1


# Preparation of Train Dataset for Surprise Models


In [5]:
# Reader 
reader = surprise.Reader(rating_scale=(-2,2))

# Build Train Set from Custom Dataset
train_set = surprise.Dataset.load_from_df(train[['CustomerID', 'StockCode', 'purchased']], reader=reader).build_full_trainset()

# Baseline Model

In [6]:
# base_model
try:
    del(base_model)
except:
    pass

# Create Object for base_model 
base_model = surprise.SVD(random_state=0, n_factors=200, n_epochs=20, verbose=True)

# Fit the base_model
base_model.fit(train_set)

Processing epoch 0
Processing epoch 1
Processing epoch 2
Processing epoch 3
Processing epoch 4
Processing epoch 5
Processing epoch 6
Processing epoch 7
Processing epoch 8
Processing epoch 9
Processing epoch 10
Processing epoch 11
Processing epoch 12
Processing epoch 13
Processing epoch 14
Processing epoch 15
Processing epoch 16
Processing epoch 17
Processing epoch 18
Processing epoch 19


# Prediction: Test Set Only

In [7]:
# Make Predictions
test_pred = predict(base_model, test, usercol='CustomerID', itemcol='StockCode')

# Show first five rows
test_pred.head()

,CustomerID,StockCode,prediction
0,17096,22766,-0.666589
1,15038,22478,0.789753
2,14684,21789,0.880522
3,12826,21034,-1.934670
4,15866,82600,-0.947074


In [9]:
# Convert Predictions
test_pred['prediction'] = np.where((test_pred['prediction']>0),1,0)

# Convert Test
test['purchased'] = np.where((test['purchased']>0),1,0)

In [10]:
# Check Distribution
test_pred['prediction'].value_counts()

0    69211
1    29190
Name: prediction, dtype: int64

In [11]:
# Sort Index of both Datasets to use Accuracy Score 
test = test.sort_values(by=['CustomerID', 'StockCode'])
test_pred = test_pred.sort_values(by=['CustomerID', 'StockCode'])

# Reset indeces for both DataFrames
test = test.reset_index(drop=True)
test_pred = test_pred.reset_index(drop=True)

In [12]:
# First Reorder columns for test 
test = test[['CustomerID', 'StockCode', 'purchased']]
# Head of Test
test.head()

,CustomerID,StockCode,purchased
0,12347,21232,0
1,12347,21754,0
2,12347,21790,0
3,12347,22086,0
4,12347,22149,0


In [13]:
# Head of Test_pred
test_pred.head()

,CustomerID,StockCode,prediction
0,12347,21232,0
1,12347,21754,0
2,12347,21790,0
3,12347,22086,0
4,12347,22149,0


In [14]:
# Import Accuracy 
from sklearn.metrics import accuracy_score

# Accuracy 
accuracy_score(test.purchased, test_pred.prediction)

0.9353868354996392

# Prediction: Top N 

In [15]:
# Predict all pairs that are not in the train set
predictions = compute_ranking_predictions(base_model, train, usercol='CustomerID', itemcol='StockCode', remove_seen=True)

In [16]:
# Check first five rows
predictions.head()

,CustomerID,StockCode,prediction
393208,14323,16169E,0.718471
393209,14323,20829,0.967231
393210,14323,20719,-0.989406
393211,14323,84030E,0.844488
393212,14323,84992,-0.781062


In [17]:
# Filter Test for purchased items only
test = test[test['purchased']==1]

In [18]:
# Now filter out top N 
# First write a function
def filter_top_n(predictions: pd.DataFrame, n: int) -> pd.DataFrame:
    
    # Group the dataframe by 'CustomerID', and for each group, sort by 'prediction' in descending order, then take the top N rows
    top_n_per_customer = predictions.groupby('CustomerID', group_keys=False).apply(lambda group: group.sort_values('prediction', ascending=False).head(n))
    
    return top_n_per_customer

# Filter Top 10 
top_10 = filter_top_n(predictions, 10)

# Filter Top 20 
top_20 = filter_top_n(predictions, 20)


In [19]:
# Evaluate Precision at 10 
eval_precision_10 = precision_at_k(test, top_10, col_user="CustomerID", col_item="StockCode",
                                    col_rating="purchased", col_prediction="prediction", 
                                    relevancy_method="top_k", k=10)
print('precision at 10 \t:', eval_precision_10)

# Evaluate Recall at 10 
eval_recall_10 = recall_at_k(test, top_10,col_user="CustomerID", col_item="StockCode",
                                    col_rating="purchased", col_prediction="prediction", 
                                    relevancy_method="top_k", k=10)
print('recall at 10 \t:', eval_recall_10)

precision at 10 	: 0.013827993254637436
recall at 10 	: 0.01213856318888441


In [20]:
# Evaluate Precision at 20 
eval_precision_20 = precision_at_k(test, top_20, col_user="CustomerID", col_item="StockCode",
                                    col_rating="purchased", col_prediction="prediction", 
                                    relevancy_method="top_k", k=20)
print('precision at 20 \t:', eval_precision_20)

# Evaluate Recall at 20 
eval_recall_20 = recall_at_k(test, top_20,col_user="CustomerID", col_item="StockCode",
                                    col_rating="purchased", col_prediction="prediction", 
                                    relevancy_method="top_k", k=20)
print('recall at 20 \t:', eval_recall_20)

precision at 20 	: 0.012462057335581788
recall at 20 	: 0.022359104261493696


# Hyper-Parameter Tuning

In [21]:
# Define Set of Hyperparameters: 
n_factors = [5, 10, 100, 200, 1000]

# Initiate List for Metrics
factors = []
recall_k = []

# Start for loop
for factor in n_factors:
    # First Delete Model Everytime
    try:
        del(model)
    except:
        pass

    # Define Model with Parameters 
    model = surprise.SVD(random_state=0, n_factors=factor, n_epochs=10)

    # Fit model
    model.fit(train_set)

    # Predict all pairs that are not in the train set
    predictions = compute_ranking_predictions(model, test, usercol='CustomerID', itemcol='StockCode')

    # Filter Top 10 
    top_10 = filter_top_n(predictions, 10)

    # Evaluate Recall at 10 
    eval_recall_10 = recall_at_k(test, top_10,col_user="CustomerID", col_item="StockCode",
                                        col_rating="purchased", col_prediction="prediction", 
                                        relevancy_method="top_k", k=10)

    print("recall at for", factor, "factors: \t", eval_recall_10)

    # Append Lists
    factors.append(factor)
    recall_k.append(eval_recall_10)

recall at for 5 factors: 	 0.01003062386023668
recall at for 10 factors: 	 0.01047891831653578
recall at for 100 factors: 	 0.010154585116788848
recall at for 200 factors: 	 0.00976204486902881
recall at for 1000 factors: 	 0.008131458112884165


# Tuning 2

In [22]:
# Define Set of Hyperparameters: 
n_factors = [2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15]

# Initiate List for Metrics
factors = []
recall_k = []

# Start for loop
for factor in n_factors:
    # First Delete Model Everytime
    try:
        del(model)
    except:
        pass

    # Define Model with Parameters 
    model = surprise.SVD(random_state=0, n_factors=factor, n_epochs=10)

    # Fit model
    model.fit(train_set)

    # Predict all pairs that are not in the train set
    predictions = compute_ranking_predictions(model, test, usercol='CustomerID', itemcol='StockCode')

    # Filter Top 10 
    top_10 = filter_top_n(predictions, 10)

    # Evaluate Recall at 10 
    eval_recall_10 = recall_at_k(test, top_10,col_user="CustomerID", col_item="StockCode",
                                        col_rating="purchased", col_prediction="prediction", 
                                        relevancy_method="top_k", k=10)

    print("recall at for", factor, "factors: \t", eval_recall_10)

    # Append Lists
    factors.append(factor)
    recall_k.append(eval_recall_10)

recall at for 2 factors: 	 0.009387703352902604
recall at for 3 factors: 	 0.009353755940222346
recall at for 4 factors: 	 0.009948360096924751
recall at for 5 factors: 	 0.01003062386023668
recall at for 6 factors: 	 0.009285557263942174
recall at for 7 factors: 	 0.01015209742895354
recall at for 8 factors: 	 0.010811180876483353
recall at for 9 factors: 	 0.010301455502454109
recall at for 10 factors: 	 0.01047891831653578
recall at for 11 factors: 	 0.010608183650314535
recall at for 12 factors: 	 0.009588871736338316
recall at for 13 factors: 	 0.00870687979646502
recall at for 14 factors: 	 0.010553471043523915
recall at for 15 factors: 	 0.01030489804990756


# Best Model: Accuracy on Test

In [23]:
# Create Object for Model
best_model = surprise.SVD(random_state=0, n_factors=8, n_epochs=20)

# Fit the base_model
best_model.fit(train_set)

# Make Predictions
test_pred = predict(best_model, test, usercol='CustomerID', itemcol='StockCode')

# Convert Predictions
test_pred['prediction'] = np.where((test_pred['prediction']>0), 1,0)

# Sort Index of both Datasets to use Accuracy Score 
test = test.sort_values(by=['CustomerID', 'StockCode'])
test_pred = test_pred.sort_values(by=['CustomerID', 'StockCode'])

# Reset indeces for both DataFrames
test = test.reset_index(drop=True)
test_pred = test_pred.reset_index(drop=True)

# Accuracy 
accuracy_score(test.purchased, test_pred.prediction)

0.8176027793862188

# Best Model: Ranking Metrics

In [24]:
# Predict All Pairs of Users & Items that are NOT in the Trainset 
predictions = compute_ranking_predictions(best_model, test, usercol='CustomerID', itemcol='StockCode')

# Filter Top 10 & 20 
top_10 = filter_top_n(predictions, 10)
top_20 = filter_top_n(predictions, 20)

# Evaluate Recall at 10 
eval_recall_10 = recall_at_k(test, top_10,col_user="CustomerID", col_item="StockCode",
                            col_rating="purchased", col_prediction="prediction", 
                            relevancy_method="top_k", k=10)
# Evaluate Recall at 20 
eval_recall_20 = recall_at_k(test, top_20,col_user="CustomerID", col_item="StockCode",
                    col_rating="purchased", col_prediction="prediction", 
                    relevancy_method="top_k", k=20)

# Evaluate Precision at 10 
eval_precision_10 = precision_at_k(test, top_10, col_user="CustomerID", col_item="StockCode",
                    col_rating="purchased", col_prediction="prediction", 
                    relevancy_method="top_k", k=10)
# Evaluate Precision at 20 
eval_precision_20 = precision_at_k(test, top_20, col_user="CustomerID", col_item="StockCode",
                    col_rating="purchased", col_prediction="prediction", 
                    relevancy_method="top_k", k=20)

print('Recall@10:\t', eval_recall_10)
print('Recall@20:\t', eval_recall_20) 
print('Precision@10:\t', eval_precision_10)
print('Precision@20:\t', eval_precision_20)

Recall@10:	 0.011747354376886236
Recall@20:	 0.021685525974064836
Precision@10:	 0.011871838111298483
Precision@20:	 0.011433389544688026
